In [1]:
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from prophet.serialize import model_to_json, model_from_json
import pandas as pd
import json
from tqdm import tqdm
from datetime import date
from llaves_more_then_q95 import llaves_more_then_q95
import numpy as np
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')
import logging

# Set the logging level for cmdstanpy to WARNING
logging.getLogger('cmdstanpy').setLevel(logging.WARNING)


In [2]:
df_grouped_by_week = pd.read_parquet('./datasets/almacenes_si_curated_by_week.parquet')

In [39]:
df_grouped_by_week[df_grouped_by_week['combination'] == '248BH7QD2T04']

,combination,date_week,quantity,discount_for_event,campaigns_name,price_taxes_excluded,campaign
298546,248BH7QD2T04,2018-05-14,9,0.0,no-discount,18400.0,0


In [37]:
    # Get the dataframe for the combination
combination = '230BX3'
df_combination = df_grouped_by_week[df_grouped_by_week['combination'] == combination]
df_combination = df_combination[['date_week','combination','quantity']]

if combination in llaves_more_then_q95 and len(df_combination[df_combination['date_week'] >= '2022-01-01']) > 0: # si la llave esta en las llaves q95 y tiene data en el 2023, se entrena con data 2023, de lo contrario, se entra con todo lo disponible
        df_combination = df_combination[df_combination['date_week'] >= '2022-01-01']
    # df_combination_test = df_grouped_by_week_test[df_grouped_by_week_test['combination'] == combination][['date_week','quantity','campaign','discount_for_event']] # get the y_true values of the combination selected


familia = str(combination[:3])
df_discount_and_campaings = archivo_campanias[archivo_campanias['familia'] == familia]

# df_discount_and_campaings = df_grouped_by_week[df_grouped_by_week['date_week'] >= '2024-01-01']
# df_discount_and_campaings = df_discount_and_campaings[['combination','date_week','discount_for_event','campaign']]
# Complete the missing weeks with 0
first_date = df_combination['date_week'].min() # Get the first date in the dataframe
last_date = df_combination['date_week'].max() # Get the last date in the dataframe


# Create a dataframe with all the weeks between the year of the first date and the year of the last date
df_dates = pd.DataFrame({'date_week': pd.date_range(start=f'{first_date.year}-01-08', end=f'2023-12-31', freq='W-MON')})    

df_combination = df_dates.merge(df_combination, on='date_week', how='left') # Merge the dataframes
df_combination['quantity'] = df_combination['quantity'].fillna(0) # Replace NaN values with 0
df_combination = df_combination[~(df_combination['combination'].isnull())]
df_combination = df_combination.merge(df_discount_and_campaings, on='date_week', how='left') # Merge the dataframes

In [38]:
df_combination

,date_week,combination,quantity,familia,discount_for_event,campaign
0,2022-10-17,230BX3,3.0,230,40.000000,1
1,2022-10-24,230BX3,8.0,230,40.000000,1
2,2022-10-31,230BX3,10.0,230,0.000000,0
3,2022-11-07,230BX3,22.0,230,0.000000,0
4,2022-11-14,230BX3,19.0,230,0.000000,0
5,2022-11-21,230BX3,30.0,230,0.000000,0
6,2022-11-28,230BX3,19.0,230,34.285713,0
7,2022-12-05,230BX3,16.0,230,32.000000,0
8,2022-12-12,230BX3,6.0,230,0.000000,0
9,2022-12-19,230BX3,5.0,230,0.000000,0


In [28]:
# Create the prophet dataframe
prophet_dataframe = pd.DataFrame()
prophet_dataframe['ds'] = df_combination['date_week']
prophet_dataframe['y'] = df_combination['quantity']
prophet_dataframe['campaign'] = df_combination['campaign']
prophet_dataframe['discount_for_event'] = df_combination['discount_for_event']
prophet_dataframe['y'] = prophet_dataframe['y'].astype(int)

n_unique_years = prophet_dataframe['ds'].dt.year.nunique() # Count the number of different years in the dataframe


In [30]:

# Create the model
if n_unique_years == 1:
    model = Prophet(weekly_seasonality = 13)
    model.add_regressor('campaign')
    model.add_regressor('discount_for_event')

else:
    model = Prophet(weekly_seasonality = 52)
    model.add_regressor('campaign')
    model.add_regressor('discount_for_event')